### Libraries

In [1]:
import pandas as pd
import os, sys 

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [3]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

### Load  requirements

In [4]:
from glob import glob

glob("prompts/*")

[]

In [5]:
with open('app/prompts/prompts_1.txt', 'r') as file:
    prompt = file.read()
    #print(conteudo)

In [6]:
file_path = "Template - Requisito de Software v02.xlsx"
#file_path = "Requisito de Produto v01.xlsx"


In [7]:
# Carregar o arquivo Excel
#file_path = "Template - Requisito de Software v02.xlsx"
xls = pd.ExcelFile(file_path)

try:
    # Carregar abas específicas
    palavra_chave = "requisito"
    resultado = [item for idx, item in enumerate(xls.sheet_names) if palavra_chave in item.lower()]
    requisito_columns=resultado[0]

    requisitos = pd.read_excel(xls, sheet_name=requisito_columns, skiprows= len(xls.sheet_names)+1)

    requisitos= requisitos.dropna(subset=['Requisito','Categoria'])

except FileNotFoundError:
    raise FileNotFoundError(f"O arquivo '{file}' não foi encontrado.")
except pd.errors.EmptyDataError:
    raise ValueError("O arquivo está vazio ou não contém dados válidos.")
except Exception as e:
    raise ValueError(f"Erro ao processar o arquivo: {e}")# Exibir as primeiras linhas da aba de requisitos
#print(requisitos.head())

requisitos_texto = requisitos.to_string(index=False)

/opt/conda/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
requisitos.head()

,Req ID,Categoria,Descrição Alto Nível,Requisito,Comentários,Status,Histórico,Versão de modificação,Origem,Jira issue,Test Case Jira
0,PRS-00001,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve ser capaz de operar em cinc...,NaN,Novo,NaN,v_1,NaN,NaN,NaN
1,PRS-00002,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
2,PRS-00003,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
3,PRS-00004,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
4,PRS-00005,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN


### Format requirements in docs

In [9]:
from langchain.docstore.document import Document

# Criar lista de documentos
docs = []
for _, row in requisitos.iterrows():
    content = f"ID: {row['Req ID']} | Categoria: {row['Categoria']} | Descrição Alto Nível: {row['Descrição Alto Nível']} | Requisito: {row['Requisito']}"
    docs.append(Document(page_content=content, metadata={"ID": row["Req ID"]}))

In [10]:
import json
docs_list =[]  
for _, row in requisitos.iterrows():
    doc = {
        "id": row['Req ID'],
        "requisito": row['Requisito']
    }
    docs_list.append(doc)

# Convertendo para JSON
docs_json = json.dumps(docs_list, indent=4, ensure_ascii=False)

### LLM

In [11]:
from langchain.llms import Ollama

llm_llama31 = Ollama(model="llama3.1:8b", temperature=0.1)
llm_llama3 = Ollama(model="llama3.3:latest", temperature=0.1)


###  Global relations analysis

In [12]:
from langchain.prompts import PromptTemplate
import json
docs_list = []
def analisar_relacoes(docs_json, llm_reqs_glob):
    prompt = """
        Analise os seguintes requisitos como um todo:
        {requisitos}

        Identifique:
        1. Redundâncias: Requisitos que se sobrepõem ou repetem informações.
        2. Dependências: Requisitos que dependem de outros para serem compreendidos ou implementados.
        3. Inconsistências: Requisitos que se contradizem ou usam terminologia inconsistente.

        Forneça uma análise detalhada e obrigatoriamente responda em português.
        """

    prompt_template = PromptTemplate(
        input_variables=["requisitos"], template=prompt)

    prompt  = prompt_template.format(requisitos=docs_json)
    analise = llm_reqs_glob(prompt)
    return analise

In [13]:
analise_global = analisar_relacoes(docs_json, llm_llama3)
print(analise_global)

/tmp/ipykernel_3969/398227033.py:21: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  analise = llm_reqs_glob(prompt)


Após analisar a lista de requisitos fornecida, identifiquei as seguintes questões:

**Redundâncias:**

1. **PRS-00329** e **PRS-00331**: Ambos os requisitos especificam dimensões físicas do dispositivo, mas com valores ligeiramente diferentes (170 mm x 75 mm x 35 mm vs. 160 mm x 65 mm). É provável que apenas um deles seja necessário.
2. **PRS-00334**, **PRS-00335** e **PRS-00336**: Esses requisitos especificam a frequência de amostragem do sinal de recepção, o número de canais físicos para transmissão e recepção, respectivamente. Embora sejam relacionados, podem ser considerados redundantes se forem parte de um mesmo sistema.
3. **PRS-00339** e **PRS-00340**: Ambos os requisitos mencionam a capacidade do dispositivo de operar com diferentes tipos de transdutores e identificar o transdutor conectado. Embora não sejam idênticos, podem ser considerados redundantes se forem parte de um mesmo sistema.

**Dependências:**

1. **PRS-00340**: Esse requisito depende da implementação do **PRS-003

### Individual Analysis

In [14]:
def avaliar_requisito(requisito, llm_reqs_indiv):
    prompt = """
    Avalie o seguinte requisito com base nas boas práticas descritas abaixo, responda em português:
    
    Boas Práticas para Requisitos:
    1. Os requisitos devem ser claros, precisos e não genéricos.
    2. Os requisitos devem ter sentenças curtas.
    3. Os requisitos devem ser individuais.
    4. Os requisitos devem ser independentes.
    5. Os requisitos devem ser consistentes.
    6. Os requisitos devem ser completos.
    7. Os requisitos devem ser verificáveis.
    8. Os requisitos não devem ser repetidos ou redundantes.
    9. Os requisitos devem ser coerentes quanto ao conteúdo.
    10. Os requisitos não devem repetir várias vezes a mesma palavra.
    
    Exemplo 1:
    Requisito: "O sistema deve permitir que o usuário faça login usando seu e-mail e senha."
    
    Avaliação:
    Clareza: Sim, o requisito é claro e específico.
    Sentenças Curtas: Sim, a sentença é curta e direta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Sim, o requisito é consistente com as boas práticas.
    Completude: Sim, o requisito descreve completamente a funcionalidade.
    Verificabilidade: Sim, é possível verificar se o login funciona com e-mail e senha.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Sim. Teste: Verificar se o usuário consegue fazer login inserindo e-mail e senha válidos.
    Classificação: Sistema (é uma funcionalidade do sistema).
    
    Exemplo 2:
    Requisito: "O sistema deve ser rápido."
    
    Avaliação:
    Clareza: Não, o requisito é vago e genérico. O que significa "rápido"?
    Sentenças Curtas: Sim, a sentença é curta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Não, o requisito é inconsistente porque "rápido" não é definido.
    Completude: Não, o requisito não descreve completamente a funcionalidade.
    Verificabilidade: Não, não é possível verificar se o sistema é "rápido" sem uma definição clara.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Não. Não há como testar algo tão subjetivo.
    Classificação: Sistema (é uma característica geral do sistema).
    
    Exemplo 3:
    Requisito: "O sistema deve enviar uma notificação ao usuário quando houver uma nova mensagem."
    
    Avaliação:
    Clareza: Sim, o requisito é claro e específico.
    Sentenças Curtas: Sim, a sentença é curta e direta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Sim, o requisito é consistente com as boas práticas.
    Completude: Sim, o requisito descreve completamente a funcionalidade.
    Verificabilidade: Sim, é possível verificar se a notificação é enviada.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Sim. Teste: Verificar se o usuário recebe uma notificação ao receber uma nova mensagem.
    Classificação: Sistema (é uma funcionalidade do sistema).
    
    Exemplo 4:
    Requisito: "O sistema deve ser fácil de usar."
    
    Avaliação:
    Clareza: Não, o requisito é vago e subjetivo. O que significa "fácil de usar"?
    Sentenças Curtas: Sim, a sentença é curta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Não, o requisito é inconsistente porque "fácil de usar" não é definido.
    Completude: Não, o requisito não descreve completamente a funcionalidade.
    Verificabilidade: Não, não é possível verificar algo tão subjetivo.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Não. Não há como testar algo tão subjetivo.
    Classificação: Produto (é uma característica geral do produto).
    
    Agora, avalie o seguinte requisito:
    
    Requisito: {requisito}
    
    Obrigatororiamente a sua resposta deve seguir o mesmo formato acima:
    
    Avaliação:
    Clareza: 
    Sentenças Curtas: 
    Individualidade: 
    Independência: 
    Consistência: 
    Completude: 
    Verificabilidade: 
    Repetição: 
    
    Testável (Identifique se o Requisito é Testável, se sim indique o teste):
    """
    prompt_template = PromptTemplate(
    input_variables=["requisito"], template=prompt)

    prompt  = prompt_template.format(requisito=requisito)
    analise = llm_reqs_indiv(prompt)
    return analise

In [20]:
prompt = """
Avalie o seguinte requisito com base nas boas práticas descritas abaixo, responda em português:

Boas Práticas para Requisitos:
1. Os requisitos devem ser claros, precisos e não genéricos.
2. Os requisitos devem ter sentenças curtas.
3. Os requisitos devem ser individuais.
4. Os requisitos devem ser independentes.
5. Os requisitos devem ser consistentes.
6. Os requisitos devem ser completos.
7. Os requisitos devem ser verificáveis.
8. Os requisitos não devem ser repetidos ou redundantes.
9. Os requisitos devem ser coerentes quanto ao conteúdo.
10. Os requisitos não devem repetir várias vezes a mesma palavra.

Exemplo 1:
Requisito: "O sistema deve permitir que o usuário faça login usando seu e-mail e senha."

Avaliação:
Clareza: Sim, o requisito é claro e específico.
Sentenças Curtas: Sim, a sentença é curta e direta.
Individualidade: Sim, trata-se de um único requisito.
Independência: Sim, este requisito não depende de outro.
Consistência: Sim, o requisito é consistente com as boas práticas.
Completude: Sim, o requisito descreve completamente a funcionalidade.
Verificabilidade: Sim, é possível verificar se o login funciona com e-mail e senha.
Repetição: Não há repetição desnecessária de palavras.

Testável: Sim. Teste: Verificar se o usuário consegue fazer login inserindo e-mail e senha válidos.
Classificação: Sistema (é uma funcionalidade do sistema).

Exemplo 2:
Requisito: "O sistema deve ser rápido."

Avaliação:
Clareza: Não, o requisito é vago e genérico. O que significa "rápido"?
Sentenças Curtas: Sim, a sentença é curta.
Individualidade: Sim, trata-se de um único requisito.
Independência: Sim, este requisito não depende de outro.
Consistência: Não, o requisito é inconsistente porque "rápido" não é definido.
Completude: Não, o requisito não descreve completamente a funcionalidade.
Verificabilidade: Não, não é possível verificar se o sistema é "rápido" sem uma definição clara.
Repetição: Não há repetição desnecessária de palavras.

Testável: Não. Não há como testar algo tão subjetivo.
Classificação: Sistema (é uma característica geral do sistema).

Exemplo 3:
Requisito: "O sistema deve enviar uma notificação ao usuário quando houver uma nova mensagem."

Avaliação:
Clareza: Sim, o requisito é claro e específico.
Sentenças Curtas: Sim, a sentença é curta e direta.
Individualidade: Sim, trata-se de um único requisito.
Independência: Sim, este requisito não depende de outro.
Consistência: Sim, o requisito é consistente com as boas práticas.
Completude: Sim, o requisito descreve completamente a funcionalidade.
Verificabilidade: Sim, é possível verificar se a notificação é enviada.
Repetição: Não há repetição desnecessária de palavras.

Testável: Sim. Teste: Verificar se o usuário recebe uma notificação ao receber uma nova mensagem.
Classificação: Sistema (é uma funcionalidade do sistema).

Exemplo 4:
Requisito: "O sistema deve ser fácil de usar."

Avaliação:
Clareza: Não, o requisito é vago e subjetivo. O que significa "fácil de usar"?
Sentenças Curtas: Sim, a sentença é curta.
Individualidade: Sim, trata-se de um único requisito.
Independência: Sim, este requisito não depende de outro.
Consistência: Não, o requisito é inconsistente porque "fácil de usar" não é definido.
Completude: Não, o requisito não descreve completamente a funcionalidade.
Verificabilidade: Não, não é possível verificar algo tão subjetivo.
Repetição: Não há repetição desnecessária de palavras.

Testável: Não. Não há como testar algo tão subjetivo.
Classificação: Produto (é uma característica geral do produto).

Agora, avalie o seguinte requisito:

Requisito: {requisito}

Obrigatororiamente a sua resposta deve seguir o mesmo formato acima:

Avaliação:
Clareza: 
Sentenças Curtas: 
Individualidade: 
Independência: 
Consistência: 
Completude: 
Verificabilidade: 
Repetição: 

Testável (Identifique se o Requisito é Testável, se sim indique o teste):
"""
prompt_template = PromptTemplate(
input_variables=["requisito"], template=prompt)

chain=prompt_template | llm_llama31
for t in chain.stream({f"requisito:{requisitos['Requisito'][0]}"}):
    print(t)

A
val
iação
:


Cl
are
za
:
 Sim
,
 o
 requis
ito
 é
 claro
 e
 específ
ico
.
 Ele
 des
cre
ve
 explicit
amente
 os
 cinco
 mod
os
 de
 oper
ação
 distint
os
 que
 o
 dispositivo
 deve
 ser
 cap
az
 de
 realizar
.


Sent
en
ças
 Curt
as
:
 Não
,
 a
 sent
ença
 é
 long
a
 e
 incl
ui
 vá
rias
 informações
.
 Emb
ora
 seja
 uma
 única
 sent
ença
,
 ela
 cont
ém
 v
ários
 detal
hes
 que
 poder
iam
 ser
 express
ados
 em
 sent
en
ças
 curt
as
 separ
adas
.


Individual
idade
:
 Sim
,
 trata
-se
 de
 um
 único
 requis
ito
 que
 des
cre
ve
 uma
 func
ional
idade
 específ
ica
 do
 dispositivo
.


Independ
ência
:
 Sim
,
 este
 requis
ito
 não
 depend
e
 de
 outro
.
 Ele
 é
 aut
ôn
omo
 e
 des
cre
ve
 uma
 caracter
ística
 independ
ente
 do
 dispositivo
.


Cons
ist
ência
:
 Sim
,
 o
 requis
ito
 é
 consist
ente
 com
 as
 bo
as
 pr
áticas
.
 Ele
 é
 claro
,
 prec
iso
 e
 não
 gen
éric
o
.


Com
plet
ude
:
 Sim
,
 o
 requis
ito
 des
cre
ve
 completamente
 a
 func
ional
idade
 que
 o
 dispositivo

In [21]:
from tqdm import tqdm

resultados_individual = []

for req in tqdm(json.loads(docs_json)):
    analise = avaliar_requisito(req['requisito'], llm_llama31)
    resultados_individual.append(analise)

# Adicionar resultados ao DataFrame
requisitos['Análise Individual'] = resultados_individual

100%|██████████| 47/47 [02:06<00:00,  2.70s/it]


In [22]:
resultados_individual[0]

'Avaliação:\n\nClareza: Sim, o requisito é claro e específico ao descrever os cinco modos de operação distintos do dispositivo.\n\nSentenças Curtas: Sim, a sentença é curta e direta, listando os modos de operação sem necessidade de explicações adicionais.\n\nIndividualidade: Sim, trata-se de um único requisito que descreve uma característica específica do dispositivo.\n\nIndependência: Sim, este requisito não depende de outro, pois descreve uma funcionalidade autônoma do dispositivo.\n\nConsistência: Sim, o requisito é consistente com as boas práticas, pois descreve claramente os modos de operação do dispositivo.\n\nCompletude: Sim, o requisito descreve completamente a funcionalidade dos modos de operação do dispositivo.\n\nVerificabilidade: Sim, é possível verificar se o dispositivo opera corretamente em cada um dos cinco modos de operação listados.\n\nRepetição: Não há repetição desnecessária de palavras ou conceitos.\n\nTestável: Sim. Teste: Verificar se o dispositivo opera corretam

In [23]:
from tqdm import tqdm

resultados_individual = []

for req in tqdm(json.loads(docs_json)):
    analise = avaliar_requisito(str(req), llm_llama31)
    resultados_individual.append(analise[analise.find('\n</think>\n')+9:])

# Adicionar resultados ao DataFrame
requisitos['Análise Individual'] = resultados_individual

100%|██████████| 47/47 [02:07<00:00,  2.71s/it]


In [ ]:
with open('analise_completa.txt', 'w') as f:
    # Análise Individual
    f.write("### Análise Individual:\n")
    for i, analise in enumerate(resultados_individual):
        Id = json.loads(docs_json)[i]['id']
        requisito =  json.loads(docs_json)[i]['requisito']
        f.write(f"ID: {id} | Requisito {requisito}: \n{analise}\n\n")

    # Análise Global
    f.write("### Análise Global:\n")
    f.write(analise_global)